In [1]:
from aoc import read_file

In [188]:
lines = read_file('20221207.txt').split('\n')

commands = [line.split(' ') for line in lines]

In [189]:
import json

def parse_commands(commands):
    i = 1

    root = {
        'name': '/',
        'path': '/',
        'type': 'dir',
        'contents': {}
    }

    dir_stack = [root]

    current_dir = root

    while i < len(lines):
        command = commands[i]
        if command[0] != '$':
            raise Exception('Should see command: {lines[index]}')
        if command[1] == 'cd':
            if command[2] == '..':
                current_dir = dir_stack.pop()
            else:
                directory_name = command[2]
                dir_stack.append(current_dir)
                current_dir = current_dir['contents'][directory_name]
        elif command[1] == 'ls':
            while i+1 < len(commands) and commands[i+1][0] != '$':
                i += 1
                if commands[i][0] == 'dir':
                    directory_name = commands[i][1]
                    current_dir['contents'][directory_name] = {
                        'name': directory_name,
                        'path': current_dir['path'] + '/' + directory_name,
                        'type': 'dir',
                        'contents': {}
                    }
                else:
                    size = int(commands[i][0])
                    file_name = commands[i][1]
                    current_dir['contents'][file_name] = {
                        'name': file_name,
                        'type': 'file',
                        'size': size
                    }
        i += 1

    return root

In [190]:
directory_tree = parse_commands(commands)

print(json.dumps(directory_tree, indent=2))

{
  "name": "/",
  "path": "/",
  "type": "dir",
  "contents": {
    "btm": {
      "name": "btm",
      "type": "file",
      "size": 282959
    },
    "fmfnpm": {
      "name": "fmfnpm",
      "path": "//fmfnpm",
      "type": "dir",
      "contents": {
        "fgtqvq": {
          "name": "fgtqvq",
          "path": "//fmfnpm/fgtqvq",
          "type": "dir",
          "contents": {
            "rjc.ncl": {
              "name": "rjc.ncl",
              "type": "file",
              "size": 293783
            },
            "wdjrhw": {
              "name": "wdjrhw",
              "type": "file",
              "size": 324635
            }
          }
        },
        "fwdvgnqp.fsm": {
          "name": "fwdvgnqp.fsm",
          "type": "file",
          "size": 194704
        },
        "fwdwq.tsq": {
          "name": "fwdwq.tsq",
          "type": "file",
          "size": 48823
        },
        "mtjngt": {
          "name": "mtjngt",
          "type": "file",
          "size

In [191]:
directory_size = {}

def get_directory_sizes(directory):
    sizes = {
        'name': directory['name'],
        'path': directory['path'],
        'contents': {}
    }
    
    dir_size = 0
    
    for name, content_file in directory['contents'].items():
        if content_file['type'] == 'file':
            dir_size += content_file['size']
        else:
            subdir_size = get_directory_sizes(content_file)
            sizes['contents'][name] = subdir_size
            dir_size += subdir_size['size']

    sizes['size'] = dir_size
    return sizes

In [192]:
directory_sizes = get_directory_sizes(directory_tree)

print(json.dumps(directory_sizes, indent=2))

{
  "name": "/",
  "path": "/",
  "contents": {
    "fmfnpm": {
      "name": "fmfnpm",
      "path": "//fmfnpm",
      "contents": {
        "fgtqvq": {
          "name": "fgtqvq",
          "path": "//fmfnpm/fgtqvq",
          "contents": {},
          "size": 618418
        },
        "rvnwwfq": {
          "name": "rvnwwfq",
          "path": "//fmfnpm/rvnwwfq",
          "contents": {},
          "size": 76914
        },
        "wrzcjwc": {
          "name": "wrzcjwc",
          "path": "//fmfnpm/wrzcjwc",
          "contents": {
            "fwdwq": {
              "name": "fwdwq",
              "path": "//fmfnpm/wrzcjwc/fwdwq",
              "contents": {},
              "size": 120795
            },
            "lddhdslh": {
              "name": "lddhdslh",
              "path": "//fmfnpm/wrzcjwc/lddhdslh",
              "contents": {
                "gzj": {
                  "name": "gzj",
                  "path": "//fmfnpm/wrzcjwc/lddhdslh/gzj",
                  "content

In [193]:
def get_directory_size_map(directory_sizes):
    size_map = {}
    
    def get_directory_size_map_rec(directory):
        size_map[directory['path']] = directory['size']
        for sub_dir in directory['contents'].values():
            get_directory_size_map_rec(sub_dir)
    
    get_directory_size_map_rec(directory_sizes)
    
    return size_map

In [194]:
size_map = get_directory_size_map(directory_sizes)

print(json.dumps(size_map, indent=2))

{
  "/": 46090134,
  "//fmfnpm": 2943317,
  "//fmfnpm/fgtqvq": 618418,
  "//fmfnpm/rvnwwfq": 76914,
  "//fmfnpm/wrzcjwc": 1510785,
  "//fmfnpm/wrzcjwc/fwdwq": 120795,
  "//fmfnpm/wrzcjwc/lddhdslh": 448360,
  "//fmfnpm/wrzcjwc/lddhdslh/gzj": 155330,
  "//fmfnpm/wrzcjwc/lddhdslh/gzj/qzgsswr": 155330,
  "//fmfnpm/wrzcjwc/mjp": 458712,
  "//fmfnpm/zlpmfh": 189296,
  "//gwlwp": 34257857,
  "//gwlwp/dwcrnbj": 3732950,
  "//gwlwp/dwcrnbj/jbtfslcn": 195255,
  "//gwlwp/dwcrnbj/lgbglc": 1409518,
  "//gwlwp/dwcrnbj/lgbglc/qdpjss": 151905,
  "//gwlwp/dwcrnbj/sbffqq": 29842,
  "//gwlwp/dwcrnbj/zhvn": 1475940,
  "//gwlwp/dwcrnbj/zhvn/clpcg": 299295,
  "//gwlwp/dwcrnbj/zhvn/gswvch": 353453,
  "//gwlwp/dwcrnbj/zhvn/gswvch/fzb": 198805,
  "//gwlwp/dwcrnbj/zhvn/gswvch/fzb/qzgsswr": 198805,
  "//gwlwp/dwcrnbj/zhvn/gswvch/mdrlrtl": 154648,
  "//gwlwp/dwcrnbj/zhvn/lgmfhnq": 303154,
  "//gwlwp/fmfnpm": 1389190,
  "//gwlwp/fmfnpm/fzb": 224479,
  "//gwlwp/fmfnpm/fzb/fzb": 224479,
  "//gwlwp/fmfnpm/fzb/fzb/gwt

In [195]:
sum([size for path, size in size_map.items() if size <= 100000])

1491614

In [196]:
available_space = 40000000

used_space = size_map['/']

required_space = used_space - available_space

for size in sorted([size for path, size in size_map.items()]):
    if size < required_space:
        continue
    else:
        print(size)
        break

6400111
